In [6]:
import utils
import pandas as pd

In [2]:
quizzes = utils.load_quizzes()

In [4]:
answers = utils.load_log('answers')

In [7]:
# Convert UTC timestamp to datetime
answers['timestamp'] = pd.to_datetime(answers['timestamp'], unit='ms')

# Only keep the latest complete answer for a given user/quiz pair
get_latest = lambda group: group.iloc[group.timestamp.argmax()]
did_complete_quiz = lambda row: len(row.answers) == len(quizzes[row.quizName]['questions'])
answers = answers \
    .loc[lambda df: df.apply(did_complete_quiz, axis=1)] \
    .groupby(['sessionId', 'quizName']) \
    .apply(get_latest) \
    .drop(columns=['sessionId', 'quizName']) \
    .reset_index()
                                                    
answers['frac_correct'] = answers.answers.map(lambda a: len([o for o in a if o['correct']]) / len(a))

In [8]:
answers.groupby('quizName').frac_correct.describe()[['count', 'mean', '50%']].sort_values('mean')

,count,mean,50%
quizName,,,
example-quiz,1.0,0.0,0.0
